In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
train_data = pd.read_csv('train.csv')

In [3]:
train_data.head(10)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.27270,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.90000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.26000,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.25300,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.30200,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1
5,5,0.105165,106,0.007932,1.276660,3165.14000,1.67015,155.5780,0.782602,-16.373700,...,5168.03000,33.917900,1.62149,1.082260,-3.972420,0.500732,0.628588,2.45744,7.23832,16
6,6,0.601966,9,-0.133924,-0.098238,734.58300,5.86411,16.6928,0.714386,-9.909540,...,-49.42170,28.144900,1.87039,0.417755,4.880680,2.884050,0.115900,1.12773,3.48813,4
7,7,0.177120,11,-0.361778,0.492327,-9.01574,6.56938,211.4200,-0.841209,16.011500,...,-5.51494,42.106500,1.79785,-0.197997,5.953100,2.695590,0.487619,1.23021,7.07088,5
8,8,0.216817,50,0.935978,-0.086906,213.43600,5.13950,136.7050,1.154870,113.220000,...,17560.20000,44.787400,1.49904,0.736440,4.647110,2.097610,0.302990,2.09165,9.73664,7
9,9,0.656791,14,-0.504919,1.547640,153.81300,6.49644,59.2214,0.619670,-37.125200,...,51.70010,12.243400,1.85891,0.061177,-2.514130,1.478190,0.513508,1.27266,2.31615,4


In [4]:
train_data.shape

(250000, 102)

In [6]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,250000.0,124999.500000,72168.927986,0.000000,62499.750000,124999.500000,187499.250000,249999.00000
f0,250000.0,0.511213,0.307884,-0.069273,0.251287,0.514962,0.777323,1.07207
f1,250000.0,51.378476,42.396636,-17.000000,18.000000,41.000000,75.000000,273.00000
f2,250000.0,0.107155,1.322200,-7.895580,-0.611172,0.253815,0.759249,9.76859
f3,250000.0,0.050010,0.792368,-1.475560,-0.719418,0.004099,0.765456,1.68019
...,...,...,...,...,...,...,...,...
f96,250000.0,2.417556,0.892563,-1.131980,1.906718,2.340430,2.910020,5.57604
f97,250000.0,0.537484,0.226589,0.005249,0.359646,0.531347,0.709807,1.10540
f98,250000.0,1.576900,0.646306,-0.646967,1.215810,1.451285,1.901633,4.49262
f99,250000.0,8.048805,5.647368,-0.842397,3.732800,7.182205,10.998550,34.01920


In [35]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Columns: 103 entries, id to kfold
dtypes: float64(95), int64(8)
memory usage: 196.5 MB


# K-fold Implementation

In [21]:
train_data['kfold'] = -1

In [25]:
train_data

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f92,f93,f94,f95,f96,f97,f98,f99,loss,kfold
0,0,-0.002350,59,0.766739,-1.350460,42.27270,16.68570,30.35990,1.267300,0.392007,...,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.296580,10.489800,15,-1
1,1,0.784462,145,-0.463845,-0.530421,27324.90000,3.47545,160.49800,0.828007,3.735860,...,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.364700,1.143830,3,-1
2,2,0.317816,19,-0.432571,-0.382644,1383.26000,19.71290,31.10260,-0.515354,34.430800,...,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.696790,12.305500,6,-1
3,3,0.210753,17,-0.616454,0.946362,-119.25300,4.08235,185.25700,1.383310,-47.521400,...,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.456450,10.028800,2,-1
4,4,0.439671,20,0.968126,-0.092546,74.30200,12.30650,72.18600,-0.233964,24.399100,...,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.784850,7.071970,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,0.923980,6,0.663212,-0.055120,6.61768,1.26619,40.44790,0.852375,25.522400,...,18.534200,1.41936,0.607504,4.950780,2.853420,0.961882,1.068010,1.437650,11,-1
249996,249996,0.243556,7,-0.557062,1.333470,-54.88610,17.58310,212.96400,-1.204750,-128.385000,...,19.482200,1.60072,-0.154877,5.693280,1.994540,0.760925,1.427880,1.226720,5,-1
249997,249997,0.046023,53,0.462863,0.704034,2062.94000,14.58160,11.42710,-0.509812,80.818000,...,11.008200,1.61080,-0.350927,27.986900,1.012690,0.830533,1.005470,0.455332,1,-1
249998,249998,0.977330,12,-1.002880,0.576377,4741.16000,11.10090,3.81546,0.616191,118.902000,...,21.985700,3.16051,0.772058,2.592220,3.938260,0.697430,0.962347,7.415890,10,-1


In [28]:
kf = model_selection.KFold(n_splits=5,shuffle=True,random_state=42)

for fold, (train_indices, valid_indices) in enumerate(kf.split(X=train_data)):
    train_data.loc[valid_indices,'kfold'] = fold


In [29]:
train_data

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f92,f93,f94,f95,f96,f97,f98,f99,loss,kfold
0,0,-0.002350,59,0.766739,-1.350460,42.27270,16.68570,30.35990,1.267300,0.392007,...,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.296580,10.489800,15,1
1,1,0.784462,145,-0.463845,-0.530421,27324.90000,3.47545,160.49800,0.828007,3.735860,...,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.364700,1.143830,3,1
2,2,0.317816,19,-0.432571,-0.382644,1383.26000,19.71290,31.10260,-0.515354,34.430800,...,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.696790,12.305500,6,3
3,3,0.210753,17,-0.616454,0.946362,-119.25300,4.08235,185.25700,1.383310,-47.521400,...,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.456450,10.028800,2,2
4,4,0.439671,20,0.968126,-0.092546,74.30200,12.30650,72.18600,-0.233964,24.399100,...,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.784850,7.071970,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,0.923980,6,0.663212,-0.055120,6.61768,1.26619,40.44790,0.852375,25.522400,...,18.534200,1.41936,0.607504,4.950780,2.853420,0.961882,1.068010,1.437650,11,4
249996,249996,0.243556,7,-0.557062,1.333470,-54.88610,17.58310,212.96400,-1.204750,-128.385000,...,19.482200,1.60072,-0.154877,5.693280,1.994540,0.760925,1.427880,1.226720,5,2
249997,249997,0.046023,53,0.462863,0.704034,2062.94000,14.58160,11.42710,-0.509812,80.818000,...,11.008200,1.61080,-0.350927,27.986900,1.012690,0.830533,1.005470,0.455332,1,0
249998,249998,0.977330,12,-1.002880,0.576377,4741.16000,11.10090,3.81546,0.616191,118.902000,...,21.985700,3.16051,0.772058,2.592220,3.938260,0.697430,0.962347,7.415890,10,1


In [43]:
test_data = pd.read_csv('test.csv')

In [44]:
useful_features = [c for c in train_data.columns if c not in ("id","kfold","loss")]

In [45]:
test_data = test_data[useful_features]

In [52]:
test_pred = []
for fold in range(5):
    xtrain = train_data[train_data['kfold'] != fold].reset_index(drop=True)
    xvalid = train_data[train_data['kfold']==fold].reset_index(drop=True)
    xtest = test_data.copy()
    
    y_train = xtrain['loss']
    y_valid = xvalid['loss']
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    sc = StandardScaler()
    xtrain[useful_features] = sc.fit_transform(xtrain[useful_features])
    xvalid[useful_features] = sc.transform(xvalid[useful_features])
    xtest[useful_features] = sc.transform(xtest[useful_features])
    
    
    model = XGBRegressor(random_state=fold, n_jobs=-1, verbose=10)
    model.fit(xtrain,y_train)
    pred_valid = model.predict(xvalid)
    pred_test = model.predict(xtest)
    test_pred.append(pred_test)
    
    print(fold, mean_squared_error(y_valid, pred_valid, squared=False))

[17:28:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0 7.900117464629388
[17:34:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


1 7.953582163022358
[17:39:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some

In [53]:
test_pred

[array([8.778026 , 5.046495 , 7.9718423, ..., 6.5864387, 5.405873 ,
        5.474859 ], dtype=float32),
 array([7.789956 , 3.8063428, 7.9986534, ..., 5.4912724, 2.4751976,
        7.303046 ], dtype=float32),
 array([ 8.087629 ,  3.6883774,  6.8200307, ...,  6.2479577, 10.167481 ,
         6.893881 ], dtype=float32),
 array([ 7.2551274,  3.917471 , 12.759374 , ...,  6.633509 ,  2.5299022,
         5.654065 ], dtype=float32),
 array([7.612997 , 3.8160312, 8.344152 , ..., 4.5424623, 4.049949 ,
        6.874454 ], dtype=float32)]

In [54]:
xtest.shape

(150000, 100)

In [56]:
np.column_stack(test_pred).shape

(150000, 5)

In [70]:
pred = np.mean(np.column_stack(test_pred), axis=1)
pred

array([7.904747 , 4.0549436, 8.7788105, ..., 5.900328 , 4.9256806,
       6.440061 ], dtype=float32)

In [61]:
sample_sub = pd.read_csv('sample_submission.csv')

In [62]:
sample_sub

,id,loss
0,250000,0
1,250001,0
2,250002,0
3,250003,0
4,250004,0
...,...,...
149995,399995,0
149996,399996,0
149997,399997,0
149998,399998,0


In [63]:
sample_sub['loss'] = pred

In [68]:
sample_sub.to_csv("sample_submissions.csv", index=False)